In [1]:
# Modelo de neurona simple con dos algoritmos de aprendizaje (SGD y Adam)

# Solo usasmosNumPy porque implementamos todo desde cero: pesos, gradientes, actualizaciones, etc.
import numpy as np

# ============================
# Funciones de activación
# ============================

# ReLU deja pasar valores positivos y convierte los negativos en 0.

# Su derivada es 1 cuando 𝑥>0 y 0 cuando 𝑥≤0.
# Se usa para introducir no linealidad en la neurona.
# Estas funciones se pasan como parámetros al modelo.

def relu(x):
    return np.maximum(0, x)

def relu_deriv(x):
    return (x > 0).astype(float)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_deriv(x):
    s = sigmoid(x)
    return s * (1 - s)

# ============================
# Clase NeuralNetwork (neurona simple)
# ============================

# Esta es la parte central del Notebook. Define cómo funciona la neurona y cómo aprende.
class NeuralNetwork:
    def __init__(self, n_inputs, n_outputs, activation, activation_deriv):
        # w: matriz de pesos inicializada aleatoriamente.
        self.w = np.random.randn(n_inputs, n_outputs) * 0.1
        # b: vector de sesgos inicializado en cero.
        self.b = np.zeros((1, n_outputs))
        self.activation = activation
        self.activation_deriv = activation_deriv

        # Parámetros Adam, inicialización
        # Estos almacenan los momentos del gradiente y el contador de pasos.
        self.m_w = np.zeros_like(self.w)
        self.v_w = np.zeros_like(self.w)
        self.m_b = np.zeros_like(self.b)
        self.v_b = np.zeros_like(self.b)
        self.t = 0

    def forward(self, X):
        # Calcula la salida lineal 𝑧=𝑋𝑊+𝑏.
        # Aplica la función de activación (ReLU por defecto).
        # Guarda z y a para el cálculo del gradiente.
        self.z = X @ self.w + self.b
        self.a = self.activation(self.z)
        return self.a

    # Cálculo de gradientes:
    # Aquí se implementa MSE (mean squared error) como función de pérdida.
    # error: diferencia entre predicción y etiqueta.
    # dz: gradiente después de la activación.
    # dw, db: gradientes de pesos y sesgos.
    # loss: pérdida promedio.
    def compute_grads(self, X, y):
        # 
        m = X.shape[0]
        error = self.a - y
        dz = error * self.activation_deriv(self.z)
        dw = (X.T @ dz) / m
        db = np.sum(dz, axis=0, keepdims=True) / m
        loss = np.mean(error**2)
        return dw, db, loss

    # ============================
    # SGD
    # ============================
    # Entrenamiento con SGD
    # Actualiza pesos usando gradiente descendente clásico.
    # Puede usar mini-batches si batch_size > 1.
    # Mezcla los datos en cada época si shuffle=True.
    # Este método es simple pero puede ser inestable.
    def train_sgd(self, X, y, epochs=200, lr=0.01):
        losses = []
        for _ in range(epochs):
            self.forward(X)
            dw, db, loss = self.compute_grads(X, y)
            self.w -= lr * dw
            self.b -= lr * db
            losses.append(loss)
        return losses

    # ============================
    # Adam
    # ============================
    # Adam es un optimizador más avanzado que ajusta automáticamente la tasa de aprendizaje.

    def train_adam(self, X, y, epochs=200, lr=0.001, beta1=0.9, beta2=0.999, eps=1e-8):
        losses = []
        for _ in range(epochs):
            self.forward(X)
            dw, db, loss = self.compute_grads(X, y)

            self.t += 1

            # Momentos
            # m_w: promedio móvil del gradiente (primer momento).
            # v_w: promedio móvil del cuadrado del gradiente (segundo momento).
            self.m_w = beta1 * self.m_w + (1 - beta1) * dw
            self.v_w = beta2 * self.v_w + (1 - beta2) * (dw**2)

            self.m_b = beta1 * self.m_b + (1 - beta1) * db
            self.v_b = beta2 * self.v_b + (1 - beta2) * (db**2)

            # Corrección de sesgo
            m_w_hat = self.m_w / (1 - beta1**self.t)
            v_w_hat = self.v_w / (1 - beta2**self.t)

            m_b_hat = self.m_b / (1 - beta1**self.t)
            v_b_hat = self.v_b / (1 - beta2**self.t)

            # Actualización final
            # Adam ajusta cada peso de forma independiente según su historial de gradientes.
            self.w -= lr * m_w_hat / (np.sqrt(v_w_hat) + eps)
            self.b -= lr * m_b_hat / (np.sqrt(v_b_hat) + eps)

            losses.append(loss)
        return losses


In [2]:
# 2. Clasificación del dataset Iris usando SGD y Adam

from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

# Cargar datos
iris = load_iris()
X = iris.data
y = iris.target.reshape(-1, 1)

# One-hot encoding
enc = OneHotEncoder(sparse_output=False)
y_onehot = enc.fit_transform(y)

# Escalado
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# División
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_onehot, test_size=0.3, random_state=42
)

# Entrenar modelos
sgd_model = NeuralNetwork(4, 3, relu, relu_deriv)
loss_sgd = sgd_model.train_sgd(X_train, y_train, epochs=300, lr=0.01)

adam_model = NeuralNetwork(4, 3, relu, relu_deriv)
loss_adam = adam_model.train_adam(X_train, y_train, epochs=300, lr=0.001)


In [3]:
# Función de precisión

def predict_classes(model, X):
    y_pred = model.forward(X)
    return np.argmax(y_pred, axis=1)

def accuracy(model, X, y_true):
    y_pred = predict_classes(model, X)
    y_true = np.argmax(y_true, axis=1)
    return np.mean(y_pred == y_true)


In [4]:
# Precisión de cada modelo

acc_sgd = accuracy(sgd_model, X_test, y_test)
acc_adam = accuracy(adam_model, X_test, y_test)

print("Precisión SGD:", acc_sgd)
print("Precisión Adam:", acc_adam)


Precisión SGD: 0.8888888888888888
Precisión Adam: 0.8666666666666667
